# **Trajectory** back on original video

IDEA:
- take the processed positions
- apply the inverse of the homography
- print on the video frame by frame all points before the current frame

In [75]:
from pathlib import Path
import cv2
import csv

import numpy as np
import pandas as pd

In [76]:
VIDEO_NUMBER = "2"
PROJECT_ROOT = Path().resolve().parent.parent
INPUT_VIDEO_PATH = str(
    PROJECT_ROOT
    / "data"
    / f"recording_{VIDEO_NUMBER}"
    / f"Recording_{VIDEO_NUMBER}.mp4"
)
TRASFORMED_CSV_PATH = str(
    PROJECT_ROOT
    / "data"
    / "auxiliary_data"
    / "reconstructed_positions"
    / f"Transformed_positions_processed_TEST_{VIDEO_NUMBER}.csv"
)
LANE_CSV_PATH = str(
    PROJECT_ROOT
    / "data"
    / "auxiliary_data"
    / "lane_points"
    / f"Lane_points_{VIDEO_NUMBER}.csv"
)

OUTPUT_VIDEO_PATH = str(
    PROJECT_ROOT
    / "data"
    / f"recording_{VIDEO_NUMBER}"
    / f"Tracked_output_TEST_{VIDEO_NUMBER}.mp4"
)
OUTPUT_CSV_PATH = str(
    PROJECT_ROOT
    / "data"
    / "auxiliary_data"
    / "circle_positions"
    / f"Ball_lower_point_raw_TEST_{VIDEO_NUMBER}.csv"
)

In [77]:
lane_df = pd.read_csv(LANE_CSV_PATH)
homographies = {}
width_px, height_px = 106, 1829  # your real-world-to-pixel scale
dst = np.array(
    [[0, height_px], [width_px, height_px], [width_px, 0], [0, 0]], dtype=np.float32
)

for _, r in lane_df.iterrows():
    frame_id = int(r["Frame"])
    src = np.array(
        [
            [r["bottom_left_x"], r["bottom_left_y"]],
            [r["bottom_right_x"], r["bottom_right_y"]],
            [r["up_right_x"], r["up_right_y"]],
            [r["up_left_x"], r["up_left_y"]],
        ],
        dtype=np.float32,
    )
    H, _ = cv2.findHomography(dst, src)
    if H is not None:
        homographies[frame_id] = H

In [78]:
positions = {}
with open(TRASFORMED_CSV_PATH, newline="") as f:
    reader = csv.reader(f)
    next(reader)  # skip header
    for frame_str, x_str, y_str in reader:
        fid = int(frame_str)
        if x_str and y_str:
            positions[fid] = (float(x_str), float(y_str))
        else:
            positions[fid] = None

In [79]:
cap = cv2.VideoCapture(str(INPUT_VIDEO_PATH))
if not cap.isOpened():
    raise RuntimeError(f"Cannot open {INPUT_VIDEO_PATH!r}")

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(str(OUTPUT_VIDEO_PATH), fourcc, fps, (w, h))

In [80]:
trajectory = []  # list of (x,y) in real-world space
last_H = None
frame_idx = 0
saved_pts = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Update homography if available
    if frame_idx in homographies:
        last_H = homographies[frame_idx]

    # Append new real-world point
    pt = positions.get(frame_idx)
    if pt is not None:
        trajectory.append(pt)

    # If we have a homography and trajectory, project & draw
    if last_H is not None and trajectory:
        # Prepare points in Nx1x2 shape
        pts_world = np.array(trajectory, dtype=np.float32).reshape(-1, 1, 2)
        pts_img = cv2.perspectiveTransform(pts_world, last_H).reshape(-1, 2)
        pts_int = np.round(pts_img).astype(np.int32)

        # Draw full trajectory polyline
        cv2.polylines(frame, [pts_int], isClosed=False, color=(0, 0, 255), thickness=2)

        # Draw most recent point as a filled circle
        x0, y0 = tuple(pts_int[-1])
        cv2.circle(frame, (x0, y0), radius=4, color=(0, 0, 255), thickness=-1)
        saved_pts.append([frame_idx, x0, y0])

    out.write(frame)
    frame_idx += 1

In [81]:
cap.release()
out.release()
print(f"Tracking video saved to: {OUTPUT_VIDEO_PATH}")

# Save the raw projected points
df_out = pd.DataFrame(saved_pts, columns=["frame", "x", "y"])
# df_out.to_csv(OUTPUT_CSV_PATH, index=False)
# print(f"Projected points saved to: {OUTPUT_CSV_PATH}")

Tracking video saved to: /home/davic/projects/IACV_project/bowling-analysis/data/recording_2/Tracked_output_TEST_2.mp4
